## Tic-Tac-Toe Agent
​
In this notebook, you will learn to build an RL agent (using Q-learning) that learns to play Numerical Tic-Tac-Toe with odd numbers. The environment is playing randomly with the agent, i.e. its strategy is to put an even number randomly in an empty cell. The following is the layout of the notebook:
        - Defining epsilon-greedy strategy
        - Tracking state-action pairs for convergence
        - Define hyperparameters for the Q-learning algorithm
        - Generating episode and applying Q-update equation
        - Checking convergence in Q-values

#### Importing libraries
Write the code to import Tic-Tac-Toe class from the environment file

In [1]:
# from <TC_Env> import <TicTacToe> - import your class from environment file
from TCGame_Env1 import TicTacToe
import collections
import numpy as np
import random
import pickle
import time
from matplotlib import pyplot as plt

In [2]:
# Function to convert state array into a string to store it as keys in the dictionary
# states in Q-dictionary will be of form: x-4-5-3-8-x-x-x-x
#   x | 4 | 5
#   ----------
#   3 | 8 | x
#   ----------
#   x | x | x

def Q_state(state):

    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [3]:
# Defining a function which will return valid (all possible actions) actions corresponding to a state
# Important to avoid errors during deployment.

def valid_actions(state):

    valid_Actions = []
    
    valid_Actions = [i for i in env.action_space(state)[0]] ###### -------please call your environment as env
    return valid_Actions

In [4]:
# Defining a function which will add new Q-values to the Q-dictionary. 
def add_to_dict(state):
    state1 = Q_state(state)
    
    valid_act = valid_actions(state)
    if state1 not in Q_dict.keys():
        for action in valid_act:
            Q_dict[state1][action]=0

#### Epsilon-greedy strategy - Write your code here

(you can build your epsilon-decay function similar to the one given at the end of the notebook)

In [5]:
# Defining epsilon-greedy policy. You can choose any function epsilon-decay strategy

def epsilon_greedy(state, time):
    max_epsilon = 1.0
    min_epsilon = 0.001
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*time)

    z = np.random.random()
    state1 = Q_state(state)
    if z > epsilon:
        action = max(Q_dict[state1],key=Q_dict[state1].get)   #Exploitation: this gets the action corresponding to max q-value of current state
    else:
        agent_actions, env_actions =  env.action_space(state) #randomly choosing an action
        # Converting to a list to get a random action from the possible actions
        list_agent_actions = list(agent_actions)
        action = tuple(list_agent_actions[random.randint(0,len(list_agent_actions) - 1)])
    return action



#### Tracking the state-action pairs for checking convergence - write your code here

In [6]:
# Initialise Q_dictionary as 'Q_dict' and States_tracked as 'States_track' (for convergence)

Q_dict = collections.defaultdict(dict)

States_track = collections.defaultdict(dict)


In [13]:
# Initialise states to be tracked
def initialise_tracking_states():
    # The states and actions should convege for the 2nd action from the agent
    sample_q_values = [('1-x-x-2-x-x-x-x-x',(2,7)),('1-2-x-x-x-x-x-x-x',(4,9)),('x-4-5-x-x-x-x-x-x',(8,3)),('x-x-x-3-8-x-x-x-x',(7,5))]
    for q_values in sample_q_values:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = [0]   #this is an array which will have appended values of that state-action pair for every 2000th episode           


In [14]:
#Defining a function to save the Q-dictionary as a pickle file

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [15]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action].append(str(Q_dict[state][action]))

In [16]:
initialise_tracking_states()

#### Define hyperparameters  ---write your code here

In [28]:
EPISODES = 150000
#EPISODES = 15000000
LR = 0.01                   #learning rate
GAMMA = 0.91


#threshold = 2000       #every these many episodes, the 4 Q-values will be stored/appended (convergence graphs)
threshold = 200
#policy_threshold = 30000    #every these many episodes, the Q-dict will be updated
policy_threshold = 300

### Q-update loop ---write your code here

In [ ]:
start_time = time.time()

for episode in range(EPISODES):
    ##### Start writing your code from the next line
    
    env = TicTacToe()      #creating an instance of TicTacToe
    
    initial_state = env.state    #for the purpose of tracking reward
    curr_state = env.state     
    
    add_to_dict(curr_state)
    
    time_step = 0
    status = False
    reward = None
    total_reward = 0  
    
    while status == False:    #the episode will run only till the TicTacToe game is not over
           
        curr_action = epsilon_greedy(curr_state, time_step)
        next_state, reward, status = env.step(curr_state, curr_action)
        
        add_to_dict(next_state)
        
        next_dict_state = Q_state(next_state)
 
        if (Q_dict[next_dict_state]):                
            # UPDATE RULE
            max_next = max(Q_dict[next_dict_state],key=Q_dict[next_dict_state].get) #this gets the action corresponding to max q-value of next state
            Q_dict[Q_state(curr_state)][curr_action] += LR * ((reward + (GAMMA*(Q_dict[Q_state(next_state)][max_next]))) - Q_dict[Q_state(curr_state)][curr_action])
            curr_state = next_state       #state(t) became state(t-1)

            total_reward += reward
        
        time_step += 1
 
    
    #TRACKING Q-VALUES
    if (episode == threshold-1):       
        initialise_tracking_states()
      
    if ((episode+1) % threshold) == 0:   
        save_tracking_states()
        save_obj(States_track,'States_tracked')   
    
    #SAVING POLICY
    if ((episode+1)% policy_threshold ) == 0:  
        save_obj(Q_dict,'Policy')    
 
    
elapsed_time = time.time() - start_time
save_obj(States_track,'States_tracked')   
save_obj(Q_dict,'Policy')

#### Check the Q-dictionary

In [ ]:
# Q_dict

In [ ]:
len(Q_dict)

In [ ]:
# try checking for one of the states - that which action your agent thinks is the best  -----This will not be evaluated
#print ('State: 1-2-4-5-x-x-x-x-x')
#print (Q_dict['1-2-4-5-x-x-x-x-x'])
#print ('State: x-4-5-3-8-x-x-x-x')
#print (Q_dict['x-4-5-3-8-x-x-x-x'])
#print ('State: 1-2-3-4-x-x-x-x-x')
#print (Q_dict['1-2-3-4-x-x-x-x-x'])
#print ('State: 1-x-x-2-5-4-x-x-x')
#print (Q_dict['1-x-x-2-5-4-x-x-x'])
Q_dict['1-x-x-2-x-x-x-x-x']
Q_dict['1-2-x-x-x-x-x-x-x']
Q_dict['x-4-5-x-x-x-x-x-x']
Q_dict['x-x-x-3-8-x-x-x-x']

#### Check the states tracked for Q-values convergence
(non-evaluative)

In [ ]:
# Write the code for plotting the graphs for state-action pairs tracked

In [ ]:
with open('States_tracked.pkl', 'rb') as handle:
    States_track = pickle.load(handle)  
print(len(States_track))
plt.figure(0, figsize=(16,7))
plt.title('state=(x-4-5-3-8-x-x-x-x) action=(7,5)')
xaxis = np.asarray(range(0, len(States_track[('x-x-x-3-8-x-x-x-x')][(7,5)])))
plt.plot(xaxis,np.asarray(States_track[('x-x-x-3-8-x-x-x-x')][(7,5)]))

### Epsilon - decay check

In [ ]:
max_epsilon = 1.0
min_epsilon = 0.001
time = np.arange(0,5000000)
epsilon = []
for i in range(0,5000000):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()